In [1]:
import os
import re
import time
import pickle
from typing import Tuple, List

import spacy
import multiprocessing
#from gensim import 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse
import gensim
from gensim.corpora import Dictionary, MmCorpus
from collections import deque
from collections import namedtuple
import random
from gensim.models.coherencemodel import CoherenceModel
# explore the topics discussed in the reviews and compare how they have evolved over time
# We will define a series of functions to achieve that objective
from gensim.models import phrases
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os

In [2]:
from smart_open import smart_open
from gensim.utils import simple_preprocess
import os
#dictionary = Dictionary(
#   simple_preprocess(line, deacc =True) for line in open('../data/pitchfork/dict.txt'))
dictionary = Dictionary.load('../data/pitchfork/dict.pkl')
bows = gensim.corpora.MmCorpus('../data/pitchfork/corpus.mm')
docs = pickle.load(open('../data/pitchfork/docs.pkl','rb'))

#dictionary = Dictionary.load_from_text('/media/data_files/github/Neural_Topic_Models/data/zhddline/dict.txt')
#bows = gensim.corpora.MmCorpus('/media/data_files/github/Neural_Topic_Models/data/zhddline/corpus.mm')
#docs = pickle.load(open('/media/data_files/github/Neural_Topic_Models/data/zhddline/docs.pkl','rb'))

In [3]:
# split into training and test
from sklearn.model_selection import train_test_split
x_bows_train, x_bows_test = train_test_split(bows, test_size=0.1, random_state=192)
x_tokens_train, x_tokens_test = train_test_split(docs, test_size=0.1, random_state=192)

In [4]:
#item = list(zip(*bows[0]))
#torch.tensor(list(item[1])).float()
print(f'len of training size is {len(x_bows_train)} and tokes are {len(x_tokens_train)} '
      f'len of test size is {len(x_bows_test)} and tokens are {len(x_tokens_test)})')

len of training size is 18782 and tokes are 18782 len of test size is 2087 and tokens are 2087)


In [5]:
from torch.utils.data import Dataset,DataLoader
import torch

class Data_Processing(object):
    def __init__(self, docs, bows, vocab):
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
        
    def __getitem__(self,idx):
        bow = torch.zeros(len(self.vocab))
        # create token and frequency
        item = list(zip(*self.bows[idx])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        # create
        bow[list(item[0])] = torch.tensor(list(item[1])).float()
        txt = self.docs[idx]
        #print(f'shape of bow before being put together in data loader {bow.shape} {type(bow)}')
        return txt, bow
    
    def __len__(self):
        return len(self.docs)
    
    def collate_fn(self,batch_data):
        texts,bows = list(zip(*batch_data))
        #print(f'what happens with collate function {torch.stack(bows,dim=0)}, {torch.stack(bows,dim=0).shape}')
        return texts,torch.stack(bows,dim=0)

    def __iter__(self):
        for doc in self.docs:
            yield doc

In [6]:
batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(x_tokens_train, x_bows_train, dictionary)
test_data = Data_Processing(x_tokens_test, x_bows_test, dictionary)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn),
                    'test': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=test_data.collate_fn)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))

In [7]:
example[1][0].shape

torch.Size([17756])

In [8]:
'''
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
from torch import nn


class Data_Processing(object):
    def __init__(self, docs, bows, vocab):
        self.docs = docs
        self.bows = bows
        self.vocab = vocab
    
# iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        # create an empty torch object to store
        #expand_array(vocab,)
        bow = np.zeros(len(self.vocab))
        item = list(zip(*self.bows[index])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]
        bow[list(item[0])] = list(item[1])
        bow = torch.tensor(bow).float()
        #bow = torch.stack(bow,dim=0)
        txt = self.docs[index]
        return txt, bow
        #return {'text':txt, 'bows':bow}
  
    def __len__(self):
        return len(self.docs)
    
    def collate_fn1(self, batch_data):
        texts, bows = list(zip(*batch_data))
        return texts, torch.stack(bows,dim=0)

batch_size = 512

# create a class to process the traininga and test data
training_data = Data_Processing(x_tokens_train, x_bows_train, dictionary)
test_data = Data_Processing(x_tokens_test, x_bows_test, dictionary)

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn1),
                    'test': DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=test_data.collate_fn1)}
dataset_sizes = {'train':len(training_data)}
example = next(iter(dataloaders_dict.get('train')))
'''

"\nfrom torch.utils.data import DataLoader\nimport torch\nimport torch.nn.functional as F\nfrom torch import nn\n\n\nclass Data_Processing(object):\n    def __init__(self, docs, bows, vocab):\n        self.docs = docs\n        self.bows = bows\n        self.vocab = vocab\n    \n# iter method to get each element at the time and tokenize it using bert        \n    def __getitem__(self, index):\n        # create an empty torch object to store\n        #expand_array(vocab,)\n        bow = np.zeros(len(self.vocab))\n        item = list(zip(*self.bows[index])) # bow = [[token_id1,token_id2,...],[freq1,freq2,...]]\n        bow[list(item[0])] = list(item[1])\n        bow = torch.tensor(bow).float()\n        #bow = torch.stack(bow,dim=0)\n        txt = self.docs[index]\n        return txt, bow\n        #return {'text':txt, 'bows':bow}\n  \n    def __len__(self):\n        return len(self.docs)\n    \n    def collate_fn1(self, batch_data):\n        texts, bows = list(zip(*batch_data))\n        re

In [9]:
example[1].shape

torch.Size([512, 17756])

In [10]:
# define the model
'''
import torch
import torch.nn.functional as F
from torch import nn

class ETM(nn.Module):
    def __init__(self, dropout=0.0,
                 num_topics = 20, vocab_size = 2000, t_hidden_size = 800, emb_dim = 300,  
                 embeddings=None, train_embeddings=True):
        super(ETM, self).__init__()
        
        self.emb_dim = emb_dim
        self.vocab_size = vocab_size
        self.num_topics = num_topics
        self.t_hidden_size = t_hidden_size
        
        # add the layers that map vocab to first hidden size
        # then add layers from hidden to t_hidden_size
        self.encoder = nn.ModuleDict({
                'enc_1': nn.Linear(vocab_size, t_hidden_size),
                'enc_2': nn.Linear(t_hidden_size, emb_dim)})
        # define mu emb_size to num_topics
        self.fc_mu = nn.Linear(emb_dim,num_topics)
        # define logvar emb_sim to num_topics
        self.fc_logvar = nn.Linear(emb_dim,num_topics)
        
        # decoder part, define layers from num_topics to hidden size
        # define layer from hidden_size to vocab size
        #self.decoder = nn.ModuleDict({
        #        'dec_1': nn.Linear(num_topics, t_hidden_size),
        #        'dec_2': nn.Linear(t_hidden_size, vocab_size)})
        
        #self.latent_dim = num_topics
        # dropout probability
        self.dropout = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(num_topics,num_topics)
        
        if train_embeddings:
            # embeddings layer
            self.rho = nn.Linear(emb_dim,vocab_size)
        else:
            pass
            # add possibility of using pretrained embeddings
            
        # embedding for topics
        self.alpha = nn.Linear(emb_dim,num_topics)
        self.decoder = None
        

    def encode(self, x):
        hid = x
        for i,layer in self.encoder.items():
            hid = F.relu(self.dropout(layer(hid)))
        mu, log_var = self.fc_mu(hid), self.fc_logvar(hid)
        return mu, log_var

    def inference(self,x):
        mu, log_var = self.encode(x)
        theta = torch.softmax(x,dim=1)
        return theta
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        hid = z
        for i,(_,layer) in enumerate(self.decoder.items()):
            hid = layer(hid)
            if i<len(self.decoder)-1:
                hid = F.relu(self.dropout(hid))
        return hid
    
    def decode1(self,z):
        wght_dec = self.alpha(self.rho.weight) #[K,V]
        #print(f'weihjt ro {wght_dec}')
        beta = F.softmax(wght_dec,dim=0).transpose(1,0)
        #print(f'beta shape {beta.shape}and beta {beta}')
        print(f'logits are z {z.shape} and beta {beta.shape}')
        res = torch.mm(z,beta)
        logits = torch.log(res+1e-6)
        return logits
    
    def forward(self, x, collate_fn=None):
        mu, log_var = self.encode(x)
        _theta = self.reparameterize(mu, log_var)
        _theta = self.fc1(_theta) 
        if collate_fn!=None:
            theta = collate_fn(_theta)
        else:
            theta = _theta
        x_reconst = self.decode1(theta)
        return x_reconst, mu, log_var
'''

"\nimport torch\nimport torch.nn.functional as F\nfrom torch import nn\n\nclass ETM(nn.Module):\n    def __init__(self, dropout=0.0,\n                 num_topics = 20, vocab_size = 2000, t_hidden_size = 800, emb_dim = 300,  \n                 embeddings=None, train_embeddings=True):\n        super(ETM, self).__init__()\n        \n        self.emb_dim = emb_dim\n        self.vocab_size = vocab_size\n        self.num_topics = num_topics\n        self.t_hidden_size = t_hidden_size\n        \n        # add the layers that map vocab to first hidden size\n        # then add layers from hidden to t_hidden_size\n        self.encoder = nn.ModuleDict({\n                'enc_1': nn.Linear(vocab_size, t_hidden_size),\n                'enc_2': nn.Linear(t_hidden_size, emb_dim)})\n        # define mu emb_size to num_topics\n        self.fc_mu = nn.Linear(emb_dim,num_topics)\n        # define logvar emb_sim to num_topics\n        self.fc_logvar = nn.Linear(emb_dim,num_topics)\n        \n        # dec

In [11]:
# VAE model
from torch import nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0):

        super(VAE, self).__init__()
        self.encoder = nn.ModuleDict({
            f'enc_{i}':nn.Linear(encode_dims[i],encode_dims[i+1]) 
            for i in range(len(encode_dims)-2)
        })
        self.fc_mu = nn.Linear(encode_dims[-2],encode_dims[-1])
        self.fc_logvar = nn.Linear(encode_dims[-2],encode_dims[-1])

        self.decoder = nn.ModuleDict({
            f'dec_{i}':nn.Linear(decode_dims[i],decode_dims[i+1])
            for i in range(len(decode_dims)-1)
        })
        self.latent_dim = encode_dims[-1]
        self.dropout = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(encode_dims[-1],encode_dims[-1])
        
        
    def encode(self, x):
        hid = x
        for i,layer in self.encoder.items():
            hid = F.relu(self.dropout(layer(hid)))
        mu, log_var = self.fc_mu(hid), self.fc_logvar(hid)
        return mu, log_var

    def inference(self,x):
        mu, log_var = self.encode(x)
        theta = torch.softmax(x,dim=1)
        return theta
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        #print('we are using original decoder of vae')
        hid = z
        for i,(_,layer) in enumerate(self.decoder.items()):
            hid = layer(hid)
            if i<len(self.decoder)-1:
                hid = F.relu(self.dropout(hid))
        return hid
    
    def forward(self, x, collate_fn=None):
        #print(f'inputs of encoder {x.shape}')
        # mu and log_var
        mu, log_var = self.encode(x)
        #print(f'shape of the outputs {mu.shape, log_var.shape}')
        _theta = self.reparameterize(mu, log_var)
        #print(f'shape of theta reparametized {_theta.shape}')
        _theta = self.fc1(_theta) 
        #print(f'shape of theta after FC1 {_theta.shape}')
        if collate_fn!=None:
            #print(f'we are collating and shape is {_theta.shape}')
            theta = collate_fn(_theta)
            #print(f'theta collated {theta.shape}')
        else:
            theta = _theta
        x_reconst = self.decode(theta)
        #print(f'output of theta from decoder {x_reconst.shape}')
        #time.sleep(5)

        return x_reconst, mu, log_var

class EVAE(VAE):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0,emb_dim=300):
        super(EVAE,self).__init__(encode_dims=encode_dims,decode_dims=decode_dims,dropout=dropout)
        self.emb_dim = emb_dim
        self.vocab_size = encode_dims[0]
        self.n_topic = encode_dims[-1]
        self.rho = nn.Linear(emb_dim,self.vocab_size)
        self.alpha = nn.Linear(emb_dim,self.n_topic)
        self.decoder = None

    def decode(self,z):
        #print('we are here enforcing the decoder we redefing')
        wght_dec = self.alpha(self.rho.weight) #[K,V]
        #print(f'we are here defining the weights of rho {wght_dec.shape}')
        beta = F.softmax(wght_dec,dim=0).transpose(1,0)
        #print(f'we are now on beta and the shape is {beta.shape}')
        res = torch.mm(z,beta)
        #print(f'res is the matmul of z which is {res.shape}')
        logits = torch.log(res+1e-6)
        #print(f'logits of the log of res are {logits.shape}')
        return logits
    

In [12]:
#model = EVAE(encode_dims=[len(dictionary),1024,512,20],decode_dims=[20,512,len(dictionary)],
#             dropout=0.0,emb_dim=300)

In [13]:
#model

In [14]:
# we now prepare a bunch of auxiliary functions to evaluate the consistency of the model
import numpy as np
from gensim.models.coherencemodel import CoherenceModel

def calc_topic_diversity(topic_words):
    '''topic_words is in the form of [[w11,w12,...],[w21,w22,...]]'''
    vocab = set(sum(topic_words,[]))
    n_total = len(topic_words) * len(topic_words[0])
    topic_div = len(vocab) / n_total
    return topic_div

def calc_topic_coherence(topic_words,docs,dictionary,emb_path=None,taskname=None,sents4emb=None,calc4each=False):
    # emb_path: path of the pretrained word2vec weights, in text format.
    # sents4emb: list/generator of tokenized sentences.
    # Computing the C_V score
    cv_coherence_model = CoherenceModel(topics=topic_words,texts=docs,dictionary=dictionary,coherence='c_v')
    cv_per_topic = cv_coherence_model.get_coherence_per_topic() if calc4each else None
    cv_score = cv_coherence_model.get_coherence()
    
    # Computing the C_W2V score
    try:
        w2v_model_path = os.path.join(os.getcwd(),'data',f'{taskname}','w2v_weight_kv.txt')
        # Priority order: 1) user's embed file; 2) standard path embed file; 3) train from scratch then store.
        if emb_path!=None and os.path.exists(emb_path):
            keyed_vectors = gensim.models.KeyedVectors.load_word2vec_format(emb_path,binary=False)
        elif os.path.exists(w2v_model_path):
            keyed_vectors = gensim.models.KeyedVectors.load_word2vec_format(w2v_model_path,binary=False)
        elif sents4emb!=None:
            print('Training a word2vec model 20 epochs to evaluate topic coherence, this may take a few minutes ...')
            w2v_model = gensim.models.Word2Vec(sents4emb,size=300,min_count=1,workers=6,iter=20)
            keyed_vectors = w2v_model.wv
            keyed_vectors.save_word2vec_format(w2v_model_path,binary=False)
        else:
            raise Exception("C_w2v score isn't available for the missing of training corpus (sents4emb=None).")
            
        w2v_coherence_model = CoherenceModel(topics=topic_words,texts=docs,dictionary=dictionary,coherence='c_w2v',keyed_vectors=keyed_vectors)

        w2v_per_topic = w2v_coherence_model.get_coherence_per_topic() if calc4each else None
        w2v_score = w2v_coherence_model.get_coherence()
    except Exception as e:
        print(e)
        #In case of OOV Error
        w2v_per_topic = [None for _ in range(len(topic_words))]
        w2v_score = None
    
    # Computing the C_UCI score
    c_uci_coherence_model = CoherenceModel(topics=topic_words,texts=docs,dictionary=dictionary,coherence='c_uci')
    c_uci_per_topic = c_uci_coherence_model.get_coherence_per_topic() if calc4each else None
    c_uci_score = c_uci_coherence_model.get_coherence()
    
    
    # Computing the C_NPMI score
    c_npmi_coherence_model = CoherenceModel(topics=topic_words,texts=docs,dictionary=dictionary,coherence='c_npmi')
    c_npmi_per_topic = c_npmi_coherence_model.get_coherence_per_topic() if calc4each else None
    c_npmi_score = c_npmi_coherence_model.get_coherence()
    return (cv_score,w2v_score,c_uci_score, c_npmi_score),(cv_per_topic,w2v_per_topic,c_uci_per_topic,c_npmi_per_topic)


def evaluate_topic_quality(topic_words, test_data, taskname=None, calc4each=False):
    
    td_score = calc_topic_diversity(topic_words)
    print(f'topic diversity:{td_score}')
    
    (c_v, c_w2v, c_uci, c_npmi),\
        (cv_per_topic, c_w2v_per_topic, c_uci_per_topic, c_npmi_per_topic) = \
        calc_topic_coherence(topic_words=topic_words, docs=test_data.docs, dictionary=test_data.dictionary,
                             emb_path=None, taskname=taskname, sents4emb=test_data, calc4each=calc4each)
    print('c_v:{}, c_w2v:{}, c_uci:{}, c_npmi:{}'.format(
        c_v, c_w2v, c_uci, c_npmi))
    scrs = {'c_v':cv_per_topic,'c_w2v':c_w2v_per_topic,'c_uci':c_uci_per_topic,'c_npmi':c_npmi_per_topic}
    if calc4each:
        for scr_name,scr_per_topic in scrs.items():
            print(f'{scr_name}:')
            for t_idx, (score, twords) in enumerate(zip(scr_per_topic, topic_words)):
                print(f'topic.{t_idx+1:>03d}: {score} {twords}')
    
    mimno_tc = mimno_topic_coherence(topic_words, test_data.docs)
    print('mimno topic coherence:{}'.format(mimno_tc))
    if calc4each:
        return (c_v, c_w2v, c_uci, c_npmi, mimno_tc, td_score), (cv_per_topic, c_w2v_per_topic, c_uci_per_topic, c_npmi_per_topic)
    else:
        return c_v, c_w2v, c_uci, c_npmi, mimno_tc, td_score
    
def inference_by_bow(model,doc_bow):
    # doc_bow: torch.tensor [vocab_size]; optional: np.array [vocab_size]
    if isinstance(doc_bow,np.ndarray):
        doc_bow = torch.from_numpy(doc_bow)
        doc_bow = doc_bow.reshape(-1,bow_dim).to(device)
        with torch.no_grad():
            mu,log_var = model.encode(doc_bow)
            mu = self.vae.fc1(mu) 
            theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()

def inference(model, doc_tokenized, dictionary,normalize=True):
    doc_bow = torch.zeros(1,len(dictionary))
    for token in doc_tokenized:
        try:
            idx = dictionary.token2id[token]
            doc_bow[0][idx] += 1.0
        except:
            print(f'{token} not in the vocabulary.')
        doc_bow = doc_bow.to(device)
        
        with torch.no_grad():
            mu,log_var = model.encode(doc_bow)
            mu = model.fc1(mu)
            if normalize:
                theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()

def get_embed(model,train_data, num=1000):
    model.eval()
    data_loader = DataLoader(train_data, batch_size=512,
                             shuffle=False, num_workers=4, collate_fn=train_data.collate_fn)
    embed_lst = []
    txt_lst = []
    cnt = 0
    for data_batch in data_loader:
        txts, bows = data_batch
        embed = inference_by_bow(bows)
        embed_lst.append(embed)
        txt_lst.append(txts)
        cnt += embed.shape[0]
        if cnt>=num:
            break
        embed_lst = np.concatenate(embed_lst,axis=0)[:num]
        txt_lst = np.concatenate(txt_lst,axis=0)[:num]
        return txt_lst, embed_lst

def get_topic_word_dist(model,normalize=True):
    model.eval()
    with torch.no_grad():
        idxes = torch.eye(model.n_topic).to(device)
        print(f'shape of idxes is {idxes.shape}')
        word_dist = model.decode(idxes)  # word_dist: [n_topic, vocab.size]
        if normalize:
            word_dist = F.softmax(word_dist,dim=1)
        return word_dist.detach().cpu().numpy()
    
def show_topic_words(model,topic_id=None,topK=15, id2token= None, dictionary=None):
        topic_words = []
        topic_idx = []
        vals_ = []
        with torch.no_grad():
            idxes = torch.eye(model.n_topic).to(device)
            print(f'shape of idxes is {idxes.shape}')
            word_dist = model.decode(idxes)
            word_dist_logs= torch.softmax(word_dist,dim=1)
            word_out = word_dist_logs.detach().cpu().numpy()
            print(torch.argmax(word_dist_logs, dim = 1))
            vals,indices = torch.topk(word_dist_logs,topK,dim=1)
            print(f'shape of vals is {vals.shape} shape of index is {indices.shape}')
            vals = vals.cpu().tolist()
            vals_.append(vals)
            indices = indices.cpu().tolist()
            topic_idx.append(indices)
            if id2token==None and dictionary!=None:
                id2token = {v:k for k,v in dictionary.token2id.items()}
            if topic_id==None:
                print('iterating over topics')
                for i in range(model.n_topic):
                    #print(i, indices, vals)
                    topic_words.append([id2token[idx] for idx in indices[i]])
                    #topic_idx.append(indices)
            else:
                topic_words.append([id2token[idx] for idx in indices[topic_id]])
        return topic_words

def load_model(model):
    model.load_state_dict(model)

In [15]:
#model = ETM(num_topics = 20, vocab_size = len(dictionary), t_hidden_size = 1024, emb_dim = 300,  
#          embeddings=None, train_embeddings=True, dropout=0.1)

#model

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

In [17]:
'''
import wandb
wandb.init(entity="jlealtru", project="ETM_runs_p")


# WandB – Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config          # Initialize config
config.batch_size = batch_size          # input batch size for training (default: 64)
config.epochs = 400             # number of epochs to train (default: 10)
config.lr = lr               # learning rate (default: 0.01)
config.no_cuda = False         # disables CUDA training
config.seed = 42               # random seed (default: 42)
config.log_interval = 20     # how many batches to wait before logging training status
config
'''


'\nimport wandb\nwandb.init(entity="jlealtru", project="ETM_runs_p")\n\n\n# WandB – Config is a variable that holds and saves hyperparameters and inputs\nconfig = wandb.config          # Initialize config\nconfig.batch_size = batch_size          # input batch size for training (default: 64)\nconfig.epochs = 400             # number of epochs to train (default: 10)\nconfig.lr = lr               # learning rate (default: 0.01)\nconfig.no_cuda = False         # disables CUDA training\nconfig.seed = 42               # random seed (default: 42)\nconfig.log_interval = 20     # how many batches to wait before logging training status\nconfig\n'

In [18]:
'''
def training(model = None, epochs = 100, optimizer = None, vocab = None):
        """
        Training the mothefucking model
        """
        acc_loss = 0
        acc_kl_theta_loss = 0
        cnt = 0
        trainloss_lst, valloss_lst = [], []
        recloss_lst, klloss_lst = [],[]
        for i in range(epochs):
            model.train()
            optimizer.zero_grad()
            #model.zero_grad()
            epochloss_lst = []
            for index, (bows) in enumerate(dataloaders_dict['train']):
                bows = bows[1].to(device)
                bows_recon,mus,log_vars = model(bows,lambda x:torch.softmax(x,dim=1))
                logsoftmax = torch.log_softmax(bows_recon,dim=1)
                #bows_recon,mus,log_vars  = model(bows)
                #print(bows_recon.shape)
                #rint(bows_recon[0])
                #model(bows, lambda x:torch.softmax(x,dim=1))
                #logsoftmax = torch.log_softmax(bows_recon, lambda x:torch.softmax(x,dim=1))
                rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                
                kl_div = -0.5 * torch.sum(1+log_vars-mus.pow(2)-log_vars.exp())
                
                loss = rec_loss + kl_div * 1.0
                
                loss.backward()
                optimizer.step()

                trainloss_lst.append(loss.item()/len(bows))
                epochloss_lst.append(loss.item()/len(bows))
                
                if index % 20==0:
                    print(f'Epoch {(i+1):>3d}\tIter {(i+1):>4d}\tLoss:{loss.item()/len(bows):<.7f}\tRec Loss:{rec_loss.item()/len(bows):<.7f}\tKL Div:{kl_div.item()/len(bows):<.7f}')
                    #wandb.log({"Epoch": i,"Train Loss": loss.item()/len(bows),"rec_loss": rec_loss.item()/len(bows)})
            
            if (i+1)%10==0:
                save_name = f'../results/ETM_models/ETM_tp{model.num_topics}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}_ep{i}_hd{model.t_hidden_size}.ckpt'
                checkpoint = {
                    "net": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "epoch": i,
                    "param": {
                        "bow_dim": len(vocab),
                        "n_topic": 25,
                        "emb_dim": 300,
                        'hidden_t': 1024
                    }
                }
                torch.save(checkpoint,save_name)
            #if (i+1)%20==0:
            #    with torch.no_grad():
            #        model.eval()
            #        for i in range(20):
            #            show_topic_words(model,topic_id=i,topK=15, dictionary=vocab)
'''

'\ndef training(model = None, epochs = 100, optimizer = None, vocab = None):\n        """\n        Training the mothefucking model\n        """\n        acc_loss = 0\n        acc_kl_theta_loss = 0\n        cnt = 0\n        trainloss_lst, valloss_lst = [], []\n        recloss_lst, klloss_lst = [],[]\n        for i in range(epochs):\n            model.train()\n            optimizer.zero_grad()\n            #model.zero_grad()\n            epochloss_lst = []\n            for index, (bows) in enumerate(dataloaders_dict[\'train\']):\n                bows = bows[1].to(device)\n                bows_recon,mus,log_vars = model(bows,lambda x:torch.softmax(x,dim=1))\n                logsoftmax = torch.log_softmax(bows_recon,dim=1)\n                #bows_recon,mus,log_vars  = model(bows)\n                #print(bows_recon.shape)\n                #rint(bows_recon[0])\n                #model(bows, lambda x:torch.softmax(x,dim=1))\n                #logsoftmax = torch.log_softmax(bows_recon, lambda x:t

In [19]:
bow_dim = len(dictionary)
n_topic = 20
emb_dim = 300
model = EVAE(encode_dims=[bow_dim,1024,512,n_topic],decode_dims=[n_topic,512,bow_dim],dropout=0.0,emb_dim=emb_dim)
print(model)
from torch.optim import Adam
lr = 1e-3
optimizer = Adam(model.parameters(), lr=lr)
optimizer

EVAE(
  (encoder): ModuleDict(
    (enc_0): Linear(in_features=17756, out_features=1024, bias=True)
    (enc_1): Linear(in_features=1024, out_features=512, bias=True)
  )
  (fc_mu): Linear(in_features=512, out_features=20, bias=True)
  (fc_logvar): Linear(in_features=512, out_features=20, bias=True)
  (decoder): None
  (dropout): Dropout(p=0.0, inplace=False)
  (fc1): Linear(in_features=20, out_features=20, bias=True)
  (rho): Linear(in_features=300, out_features=17756, bias=True)
  (alpha): Linear(in_features=300, out_features=20, bias=True)
)


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [20]:
import wandb
wandb.init(entity="jlealtru", project="ETM_runs_p",)

wandb: Currently logged in as: jlealtru (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [21]:
def train(model,batch_size=256,
          learning_rate=2e-3,test_data=None,
          num_epochs=200,is_evaluate=False,log_every=10,beta=1.0, 
          criterion='cross_entropy',ckpt=None):
    
    model.to(device)
    model.train()
    
    #self.id2token = {v:k for k,v in train_data.dictionary.token2id.items()}
    #data_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,
    #                         num_workers=4,collate_fn=train_data.collate_fn)

    data_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4,
                                        collate_fn=training_data.collate_fn)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    if ckpt:
        self.load_model(ckpt["net"])
        optimizer.load_state_dict(ckpt["optimizer"])
        start_epoch = ckpt["epoch"] + 1
    else:
        start_epoch = 0

    trainloss_lst, valloss_lst = [], []
    recloss_lst, klloss_lst = [],[]
    c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
    for epoch in range(start_epoch, num_epochs):
        epochloss_lst = []
        for iter_,data in enumerate(data_loader):
            optimizer.zero_grad()

            txts,bows = data
            bows = bows.to(device)
            #print(f'we now are in training and the shape of bows is {bows.shape}')
            
            bows_recon,mus,log_vars = model(bows, lambda x:torch.softmax(x,dim=1))
            #print(f'we now look at output of vae with lambda x shape of bows_recons is {bows_recon.shape} mus {mus.shape} and log vars {log_vars.shape}')
            #print(bows_recon[0][0:10])

            #print(f'bows recon shape is {bows_recon.shape}')
            #print(f'sum of torch {torch.sum(bows_recon, dim = 1)}')
            #print(f'one bow is {bows_recon[0]}')
            #bows_recon = torch.softmax(bows_recon,dim=1)
            
            #print(model(bows,lambda x:torch.softmax(x,dim=1)))
            #print(model(bows)[2].shape)
            #bows_recon,mus,log_vars = model(bows)
            #print(bows_recon[1].shape)
            if criterion=='cross_entropy':
                logsoftmax = torch.log_softmax(bows_recon,dim=1)
                #print(f'softmax shape is {logsoftmax.shape}')
                #print(f'softmax is {logsoftmax[0]}')
                #print(f'softmax is {logsoftmax[0]}')
                rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                #print(f'rec_loss is {rec_loss}')
                #print(f'mat mul is  {torch.sum(bows*logsoftmax)}')
                #print(f'mat mul is  {torch.sum(bows*logsoftmax).shape}')
            elif criterion=='bce_softmax':
                rec_loss = F.binary_cross_entropy(torch.softmax(bows_recon,dim=1),bows,reduction='sum')
            elif criterion=='bce_sigmoid':
                rec_loss = F.binary_cross_entropy(torch.sigmoid(bows_recon),bows,reduction='sum')

            kl_div = -0.5 * torch.sum(1+log_vars-mus.pow(2)-log_vars.exp())

            loss = rec_loss + kl_div * beta

            loss.backward()
            optimizer.step()
            
            trainloss_lst.append(loss.item()/len(bows))
            epochloss_lst.append(loss.item()/len(bows))
            #print(iter_, type(epoch))
            if (iter_+1) % 6==0:
                print(f'Epoch {(epoch+1):>3d}\tIter {(iter_+1):>4d}\tLoss:{loss.item()/len(bows):<.7f}\tRec Loss:{rec_loss.item()/len(bows):<.7f}\tKL Div:{kl_div.item()/len(bows):<.7f}')
                wandb.log({"Epoch": iter_+1,
                           "Train Loss": loss.item()/len(bows),"rec_loss": rec_loss.item()/len(bows),
                          "lr": learning_rate,
                          "optimizer": 'Adam'})
        
        if (epoch+1) % 40==0:    
            print(f'Epoch {(epoch+1):>3d}\tLoss:{sum(epochloss_lst)/len(epochloss_lst):<.7f}')
            print('\n'.join([str(lst) for lst in show_topic_words(model,
                                                                  topic_id=None,
                                                                  topK=15, dictionary=dictionary)]))
            top_ = show_topic_words(model,topic_id=None,topK=15, dictionary=dictionary)
            print(calc_topic_diversity(show_topic_words(model,topic_id=None,topK=15, dictionary=dictionary)))
            print('calculating topic dist')
            size = get_topic_word_dist(model,normalize=True)
            for i in np.argsort(size[0], axis = 0)[::-1][:15]:
                print(dictionary[i])
            c_uci_coherence_model = CoherenceModel(topics=top_,texts=x_tokens_test,dictionary=dictionary,coherence='c_uci')
            c_npmi_coherence_model = CoherenceModel(topics=top_,texts=x_tokens_test,dictionary=dictionary,coherence='c_npmi')
            c_cv_coherence_model = CoherenceModel(topics=top_,texts=x_tokens_test,dictionary=dictionary,coherence='c_v')
            uci = c_uci_coherence_model.get_coherence()
            print(f'coherence c_uci {uci}')
            print(f'coherence c_npmi {c_npmi_coherence_model.get_coherence()}')
            print(f'coherence c_cv {c_cv_coherence_model.get_coherence()}')
            wandb.log({"topic_diversity": calc_topic_diversity(show_topic_words(model,topic_id=None,topK=15, dictionary=dictionary)), 
                       "c_uci": uci,"c_npmi": c_npmi_coherence_model.get_coherence(), 
                       "c_cv": c_cv_coherence_model.get_coherence(),  "rec_loss": rec_loss.item()/len(bows),
                      "epoch_loss":sum(epochloss_lst)/len(epochloss_lst)})

In [22]:
model.to(device)
wandb.watch(model, log="all")
#training(model = model, epochs=60, optimizer=optimizer, vocab = dictionary)
train(model,batch_size=1024,
      learning_rate=9e-4,test_data=None,
      num_epochs=1000,is_evaluate=False,log_every=40,beta=1.0, 
      criterion='cross_entropy',ckpt=None)

/home/jlealtru/anaconda3/envs/torch/lib/python3.7/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch   1	Iter    6	Loss:2435.6679688	Rec Loss:2435.6616211	KL Div:0.0063035
Epoch   1	Iter   12	Loss:2364.9091797	Rec Loss:2364.8942871	KL Div:0.0148828
Epoch   1	Iter   18	Loss:2432.1032715	Rec Loss:2432.0595703	KL Div:0.0436287
Epoch   2	Iter    6	Loss:2450.7968750	Rec Loss:2450.6955566	KL Div:0.1012335
Epoch   2	Iter   12	Loss:2359.8413086	Rec Loss:2359.6572266	KL Div:0.1841140
Epoch   2	Iter   18	Loss:2358.7370605	Rec Loss:2358.3994141	KL Div:0.3375591
Epoch   3	Iter    6	Loss:2285.2260742	Rec Loss:2284.5649414	KL Div:0.6611623
Epoch   3	Iter   12	Loss:2291.8762207	Rec Loss:2290.6474609	KL Div:1.2288318
Epoch   3	Iter   18	Loss:2284.7360840	Rec Loss:2282.8945312	KL Div:1.8415115
Epoch   4	Iter    6	Loss:2221.6940918	Rec Loss:2219.4140625	KL Div:2.2801003
Epoch   4	Iter   12	Loss:2207.5747070	Rec Loss:2205.4396973	KL Div:2.1349330
Epoch   4	Iter   18	Loss:2211.2480469	Rec Loss:2209.6733398	KL Div:1.5748149
Epoch   5	Iter    6	Loss:2194.7678223	Rec Loss:2193.8752441	KL Div:0.8925794

Epoch  36	Iter   18	Loss:2183.5251465	Rec Loss:2181.3105469	KL Div:2.2145827
Epoch  37	Iter    6	Loss:2239.6206055	Rec Loss:2237.4179688	KL Div:2.2026343
Epoch  37	Iter   12	Loss:2153.5051270	Rec Loss:2151.2211914	KL Div:2.2838852
Epoch  37	Iter   18	Loss:2124.1706543	Rec Loss:2121.7749023	KL Div:2.3956654
Epoch  38	Iter    6	Loss:2180.9082031	Rec Loss:2178.4326172	KL Div:2.4755754
Epoch  38	Iter   12	Loss:2183.3378906	Rec Loss:2180.8061523	KL Div:2.5317082
Epoch  38	Iter   18	Loss:2106.7446289	Rec Loss:2104.2968750	KL Div:2.4478559
Epoch  39	Iter    6	Loss:2155.3757324	Rec Loss:2152.7719727	KL Div:2.6038165
Epoch  39	Iter   12	Loss:2115.5036621	Rec Loss:2113.1035156	KL Div:2.4001021
Epoch  39	Iter   18	Loss:2173.7990723	Rec Loss:2171.3398438	KL Div:2.4592397
Epoch  40	Iter    6	Loss:2171.4216309	Rec Loss:2168.7556152	KL Div:2.6661184
Epoch  40	Iter   12	Loss:2165.2583008	Rec Loss:2162.4218750	KL Div:2.8364954
Epoch  40	Iter   18	Loss:2147.6325684	Rec Loss:2145.1354980	KL Div:2.4971211

Epoch  54	Iter   18	Loss:2167.5783691	Rec Loss:2163.7221680	KL Div:3.8562150
Epoch  55	Iter    6	Loss:2179.6130371	Rec Loss:2175.9311523	KL Div:3.6818950
Epoch  55	Iter   12	Loss:2181.6918945	Rec Loss:2177.6726074	KL Div:4.0193920
Epoch  55	Iter   18	Loss:2159.2434082	Rec Loss:2155.7500000	KL Div:3.4935269
Epoch  56	Iter    6	Loss:2158.6513672	Rec Loss:2154.7773438	KL Div:3.8739166
Epoch  56	Iter   12	Loss:2179.3840332	Rec Loss:2175.5737305	KL Div:3.8102641
Epoch  56	Iter   18	Loss:2166.4609375	Rec Loss:2162.7321777	KL Div:3.7287478
Epoch  57	Iter    6	Loss:2169.1457520	Rec Loss:2165.3583984	KL Div:3.7874174
Epoch  57	Iter   12	Loss:2196.3056641	Rec Loss:2192.3466797	KL Div:3.9590349
Epoch  57	Iter   18	Loss:2112.6333008	Rec Loss:2109.0705566	KL Div:3.5627234
Epoch  58	Iter    6	Loss:2175.6884766	Rec Loss:2171.9020996	KL Div:3.7863455
Epoch  58	Iter   12	Loss:2135.4011230	Rec Loss:2131.6784668	KL Div:3.7225504
Epoch  58	Iter   18	Loss:2125.6176758	Rec Loss:2121.9086914	KL Div:3.7090657

coherence c_uci -0.06046248307067775
coherence c_npmi -0.0033205613730262775
coherence c_cv 0.3027161108544133
shape of idxes is torch.Size([20, 20])
tensor([13238,    97,    95, 13238,   526, 13238, 13238, 13238, 13238,    97,
        13238,    97, 13238, 13238,   751,  1295, 13238, 13238,   131, 13238],
       device='cuda:0')
shape of vals is torch.Size([20, 15]) shape of index is torch.Size([20, 15])
iterating over topics
Epoch  81	Iter    6	Loss:2141.6706543	Rec Loss:2136.5075684	KL Div:5.1631513
Epoch  81	Iter   12	Loss:2116.7944336	Rec Loss:2111.9721680	KL Div:4.8223066
Epoch  81	Iter   18	Loss:2130.6413574	Rec Loss:2125.5720215	KL Div:5.0693836
Epoch  82	Iter    6	Loss:2171.9548340	Rec Loss:2166.8662109	KL Div:5.0887318
Epoch  82	Iter   12	Loss:2137.0268555	Rec Loss:2132.0505371	KL Div:4.9763222
Epoch  82	Iter   18	Loss:2175.9208984	Rec Loss:2170.5456543	KL Div:5.3751926
Epoch  83	Iter    6	Loss:2127.7221680	Rec Loss:2122.6359863	KL Div:5.0862350
Epoch  83	Iter   12	Loss:2165.8

...

coherence c_uci -0.026937966832673944
coherence c_npmi 0.012968598688221816
coherence c_cv 0.386699468433555
shape of idxes is torch.Size([20, 20])
tensor([  253,    97,    95,  2218,  1114,   235, 13238,   573,    97,   131,
          254,   317,  1654,   823,   751,  1295,   433,    99,  3078,    97],
       device='cuda:0')
shape of vals is torch.Size([20, 15]) shape of index is torch.Size([20, 15])
iterating over topics
Epoch 881	Iter    6	Loss:2146.0065918	Rec Loss:2136.9497070	KL Div:9.0569210
Epoch 881	Iter   12	Loss:2147.4416504	Rec Loss:2138.3847656	KL Div:9.0568695
Epoch 881	Iter   18	Loss:2121.7482910	Rec Loss:2112.6616211	KL Div:9.0866680
Epoch 882	Iter    6	Loss:2155.1293945	Rec Loss:2146.1794434	KL Div:8.9500141
Epoch 882	Iter   12	Loss:2128.8925781	Rec Loss:2119.7502441	KL Div:9.1422482
Epoch 882	Iter   18	Loss:2145.7756348	Rec Loss:2136.8315430	KL Div:8.9439983
Epoch 883	Iter    6	Loss:2100.0227051	Rec Loss:2091.0034180	KL Div:9.0193710
Epoch 883	Iter   12	Loss:2113.319

Epoch 914	Iter   18	Loss:2109.4338379	Rec Loss:2100.4948730	KL Div:8.9389439
Epoch 915	Iter    6	Loss:2125.5185547	Rec Loss:2116.4697266	KL Div:9.0487518
Epoch 915	Iter   12	Loss:2133.3063965	Rec Loss:2124.4467773	KL Div:8.8595963
Epoch 915	Iter   18	Loss:2100.0693359	Rec Loss:2090.9174805	KL Div:9.1519032
Epoch 916	Iter    6	Loss:2095.9248047	Rec Loss:2087.0771484	KL Div:8.8476067
Epoch 916	Iter   12	Loss:2111.9416504	Rec Loss:2102.7099609	KL Div:9.2316380
Epoch 916	Iter   18	Loss:2108.3078613	Rec Loss:2099.5227051	KL Div:8.7852058
Epoch 917	Iter    6	Loss:2113.4848633	Rec Loss:2104.4340820	KL Div:9.0507298
Epoch 917	Iter   12	Loss:2136.0070801	Rec Loss:2126.8825684	KL Div:9.1244373
Epoch 917	Iter   18	Loss:2160.8076172	Rec Loss:2151.9106445	KL Div:8.8970013
Epoch 918	Iter    6	Loss:2081.5146484	Rec Loss:2072.6132812	KL Div:8.9013796
Epoch 918	Iter   12	Loss:2088.9797363	Rec Loss:2080.0017090	KL Div:8.9780388
Epoch 918	Iter   18	Loss:2144.7807617	Rec Loss:2135.8403320	KL Div:8.9404879

Epoch 931	Iter   18	Loss:2127.4479980	Rec Loss:2118.5590820	KL Div:8.8889809
Epoch 932	Iter    6	Loss:2115.2275391	Rec Loss:2106.1176758	KL Div:9.1099586
Epoch 932	Iter   12	Loss:2147.8317871	Rec Loss:2138.8220215	KL Div:9.0098467
Epoch 932	Iter   18	Loss:2119.7575684	Rec Loss:2110.7314453	KL Div:9.0261898
Epoch 933	Iter    6	Loss:2123.9973145	Rec Loss:2114.9262695	KL Div:9.0710831
Epoch 933	Iter   12	Loss:2144.1606445	Rec Loss:2135.2358398	KL Div:8.9248695
Epoch 933	Iter   18	Loss:2137.3083496	Rec Loss:2128.3569336	KL Div:8.9514685
Epoch 934	Iter    6	Loss:2063.8818359	Rec Loss:2055.0092773	KL Div:8.8726501
Epoch 934	Iter   12	Loss:2112.3928223	Rec Loss:2103.4741211	KL Div:8.9185925
Epoch 934	Iter   18	Loss:2149.0537109	Rec Loss:2139.8886719	KL Div:9.1649513
Epoch 935	Iter    6	Loss:2094.3869629	Rec Loss:2085.4877930	KL Div:8.8991632
Epoch 935	Iter   12	Loss:2130.5161133	Rec Loss:2121.4272461	KL Div:9.0889893
Epoch 935	Iter   18	Loss:2152.7751465	Rec Loss:2143.7260742	KL Div:9.0489521

coherence c_uci -0.08450339092662869
coherence c_npmi 0.010984092188086922
coherence c_cv 0.38886585336410245
shape of idxes is torch.Size([20, 20])
tensor([  253,    97,   454,  2218,  1114,   235, 13238,   609,  1307,   131,
          254,   571,  1654,  1402,   751,  1295,   433,    99,  3078,    97],
       device='cuda:0')
shape of vals is torch.Size([20, 15]) shape of index is torch.Size([20, 15])
iterating over topics
Epoch 961	Iter    6	Loss:2147.3894043	Rec Loss:2138.2827148	KL Div:9.1067734
Epoch 961	Iter   12	Loss:2157.0786133	Rec Loss:2148.0241699	KL Div:9.0544720
Epoch 961	Iter   18	Loss:2119.5297852	Rec Loss:2110.5820312	KL Div:8.9477615
Epoch 962	Iter    6	Loss:2168.5114746	Rec Loss:2159.5053711	KL Div:9.0060902
Epoch 962	Iter   12	Loss:2142.3122559	Rec Loss:2133.3632812	KL Div:8.9488907
Epoch 962	Iter   18	Loss:2135.8818359	Rec Loss:2126.9743652	KL Div:8.9075451
Epoch 963	Iter    6	Loss:2094.0417480	Rec Loss:2085.1442871	KL Div:8.8974571
Epoch 963	Iter   12	Loss:2126.50

Epoch 994	Iter   18	Loss:2146.9338379	Rec Loss:2138.0710449	KL Div:8.8628283
Epoch 995	Iter    6	Loss:2118.4912109	Rec Loss:2109.3779297	KL Div:9.1133080
Epoch 995	Iter   12	Loss:2149.4191895	Rec Loss:2140.4875488	KL Div:8.9316139
Epoch 995	Iter   18	Loss:2111.7099609	Rec Loss:2102.7214355	KL Div:8.9885540
Epoch 996	Iter    6	Loss:2094.9948730	Rec Loss:2086.1069336	KL Div:8.8880291
Epoch 996	Iter   12	Loss:2157.1257324	Rec Loss:2147.9526367	KL Div:9.1730833
Epoch 996	Iter   18	Loss:2148.9843750	Rec Loss:2139.9819336	KL Div:9.0024853
Epoch 997	Iter    6	Loss:2153.3674316	Rec Loss:2144.2421875	KL Div:9.1252251
Epoch 997	Iter   12	Loss:2123.2509766	Rec Loss:2114.3432617	KL Div:8.9076433
Epoch 997	Iter   18	Loss:2126.5507812	Rec Loss:2117.5507812	KL Div:8.9999790
Epoch 998	Iter    6	Loss:2113.6635742	Rec Loss:2104.7333984	KL Div:8.9302464
Epoch 998	Iter   12	Loss:2170.6987305	Rec Loss:2161.5666504	KL Div:9.1320419
Epoch 998	Iter   18	Loss:2174.1992188	Rec Loss:2165.3405762	KL Div:8.8586655

In [23]:
#topic_words = show_topic_words(model,topic_id=None,topK=15, dictionary=dictionary)
cv_coherence_model = CoherenceModel(topics=topic_words,texts=x_tokens_train,dictionary=dictionary,coherence='c_uci')
cv_coherence_model.get_coherence()
#CoherenceModel(model=model, corpus=docs, coherence='c_v', dictionary = dictionary)

NameError: name 'topic_words' is not defined

In [ ]:
nb

In [ ]:
#size = get_topic_word_dist(model,normalize=True)
#size.shape
for i in np.argsort(size[0], axis = 0)[::-1][:15]:
    print(dictionary[i])

In [ ]:
#a = show_topic_words(model, topic_id = None, topK=15, id2token=None, dictionary=dictionary)
#np.argsort(a[1][0].detach().cpu().numpy(),axis=0)
#dictionary[2336]#id2token = {v:k for k,v in dictionary.token2id.items()}
#np.argsort(a[1][0].detach().cpu().numpy(),axis=0)[0]
assert a[1][0][5405]<a[1][0][2336]

In [ ]:
get_topic_words(model,topn=15,n_topic=20, vocab=None,fix_topic=None, showWght=False):
    # define topic list
    topics = []
    def show_one_tp(tp_idx):
        if showWght:
            return [(vocab.id2token[t[0]],t[1]) for t in model.get_topic_terms(tp_idx,topn=topn)]
        else:
            return [vocab.id2token[t[0]] for t in model.get_topic_terms(tp_idx,topn=topn)]
    if fix_topic is None:
        for i in range(n_topic):
            topics.append(show_one_tp(i))
    else:
        topics.append(show_one_tp(fix_topic))
    return topics

In [ ]:
def get_topic_words(model,topn=15,n_topic=10,vocab=None,fix_topic=None,showWght=False):
    topics = []
    def show_one_tp(tp_idx):
        if showWght:
            return [(vocab.id2token[t[0]],t[1]) for t in model.get_topic_terms(tp_idx,topn=topn)]
        else:
            return [vocab.id2token[t[0]] for t in model.get_topic_terms(tp_idx,topn=topn)]
    if fix_topic is None:
        for i in range(n_topic):
            topics.append(show_one_tp(i))
    else:
        topics.append(show_one_tp(fix_topic))
    return topics
get_topic_words(model, topn=15, n_topic=20, vocab=dictionary,fix_topic=None,showWght=True)

In [ ]:
for i in range(20):
    print(show_topic_words(model,topic_id=i,topK=15, dictionary=dictionary))

In [ ]:
a
get_topics(model = mod, num_topics = 25, top_n_words= 5, vocabulary = dictionary)

In [ ]:
with torch.no_grad():
    topics = []
    gammas = model.get_beta()
    for k in range(25):
        gamma = gammas[k]
        top_words = list(gamma.cpu().numpy().argsort())
        topic_words = [vocab[a] for a in top_words]
        topics.append(topic_words)

In [ ]:
gammas[3].cpu().numpy().argsort()
[vocab[a] for a in gammas[2].cpu().numpy().argsort()][-10:-1]
topics_words = []
top_words = list(gammas[2].cpu().numpy().argsort()[-5+1:][::-1])
topic_words = [vocab[a].strip() for a in top_words]
topics_words.append(' '.join(topic_words))
topics_words


In [ ]:

def visualize(self, args, vocabulary, show_emb=False):
    Path.cwd().joinpath("results").mkdir(parents=True, exist_ok=True)
    self.eval()
    model_path = Path.home().joinpath("Projects", 
                                        "Personal", 
                                        "balobi_nini", 
                                        'models', 
                                        'embeddings_one_gram_fast_tweets_only').__str__()
    model_gensim = FT_gensim.load(model_path)

        # need to update this .. 
    queries = ['jazz','rock','radiohead']

        ## visualize topics using monte carlo
    with torch.no_grad():
        print('#'*100)
        print('Visualize topics...')
        topics_words = []
        gammas = self.get_beta()
        for k in range(args.num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[-args.num_words+1:][::-1])
            topic_words = [vocabulary[a].strip() for a in top_words]
            topics_words.append(' '.join(topic_words))
            if show_emb:
                ## visualize word embeddings by using V to get nearest neighbors
                print('#'*100)
                print('Visualize word embeddings by using output embedding matrix')
                try:
                    embeddings = model.rho.weight  # Vocab_size x E
                except:
                    embeddings = self.rho         # Vocab_size x E
                neighbors = []
                for word in queries:
                    print('word: {} .. neighbors: {}'.format(
                        word, nearest_neighbors(model_gensim, word)))
                print('#'*100)


In [ ]:
import os
import re
import time
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import codecs

class VAE(nn.Module):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0):

        super(VAE, self).__init__()
        self.encoder = nn.ModuleDict({
            f'enc_{i}':nn.Linear(encode_dims[i],encode_dims[i+1]) 
            for i in range(len(encode_dims)-2)
        })
        self.fc_mu = nn.Linear(encode_dims[-2],encode_dims[-1])
        self.fc_logvar = nn.Linear(encode_dims[-2],encode_dims[-1])

        self.decoder = nn.ModuleDict({
            f'dec_{i}':nn.Linear(decode_dims[i],decode_dims[i+1])
            for i in range(len(decode_dims)-1)
        })
        self.latent_dim = encode_dims[-1]
        self.dropout = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(encode_dims[-1],encode_dims[-1])
        
        
    def encode(self, x):
        hid = x
        for i,layer in self.encoder.items():
            hid = F.relu(self.dropout(layer(hid)))
        mu, log_var = self.fc_mu(hid), self.fc_logvar(hid)
        return mu, log_var

    def inference(self,x):
        mu, log_var = self.encode(x)
        theta = torch.softmax(x,dim=1)
        return theta
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        hid = z
        for i,(_,layer) in enumerate(self.decoder.items()):
            hid = layer(hid)
            if i<len(self.decoder)-1:
                hid = F.relu(self.dropout(hid))
        return hid
    
    def forward(self, x, collate_fn=None):
        mu, log_var = self.encode(x)
        _theta = self.reparameterize(mu, log_var)
        _theta = self.fc1(_theta) 
        if collate_fn!=None:
            theta = collate_fn(_theta)
        else:
            theta = _theta
        x_reconst = self.decode(theta)
        return x_reconst, mu, log_var

In [ ]:
class EVAE(VAE):
    def __init__(self, encode_dims=[2000,1024,512,20],decode_dims=[20,1024,2000],dropout=0.0,emb_dim=300):
        super(EVAE,self).__init__(encode_dims=encode_dims,decode_dims=decode_dims,dropout=dropout)
        self.emb_dim = emb_dim
        self.vocab_size = encode_dims[0]
        self.n_topic = encode_dims[-1]
        self.rho = nn.Linear(emb_dim,self.vocab_size)
        self.alpha = nn.Linear(emb_dim,self.n_topic)
        self.decoder = None

    def decode(self,z):
        wght_dec = self.alpha(self.rho.weight) #[K,V]
        beta = F.softmax(wght_dec,dim=0).transpose(1,0)
        res = torch.mm(z,beta)
        logits = torch.log(res+1e-6)
        return logits


class ETM:
    def __init__(self,bow_dim=10000,n_topic=20,taskname=None,device=None,emb_dim=300):
        self.bow_dim = bow_dim
        self.n_topic = n_topic
        self.emb_dim = emb_dim
        #TBD_fc1
        self.vae = EVAE(encode_dims=[bow_dim,1024,512,n_topic],decode_dims=[n_topic,512,bow_dim],dropout=0.0,emb_dim=emb_dim)
        self.device = device
        self.id2token = None
        self.taskname = taskname
        if device!=None:
            self.vae = self.vae.to(device)

    def train(self,train_data,batch_size=256,learning_rate=1e-3,test_data=None,num_epochs=100,is_evaluate=False,log_every=5,beta=1.0,criterion='cross_entropy',ckpt=None):
        self.vae.train()
        self.id2token = {v:k for k,v in train_data.dictionary.token2id.items()}
        data_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=4,collate_fn=train_data.collate_fn)

        optimizer = torch.optim.Adam(self.vae.parameters(),lr=learning_rate)
        #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        if ckpt:
            self.load_model(ckpt["net"])
            optimizer.load_state_dict(ckpt["optimizer"])
            start_epoch = ckpt["epoch"] + 1
        else:
            start_epoch = 0

        trainloss_lst, valloss_lst = [], []
        recloss_lst, klloss_lst = [],[]
        c_v_lst, c_w2v_lst, c_uci_lst, c_npmi_lst, mimno_tc_lst, td_lst = [], [], [], [], [], []
        for epoch in range(start_epoch, num_epochs):
            epochloss_lst = []
            for iter,data in enumerate(data_loader):
                optimizer.zero_grad()

                txts,bows = data
                bows = bows.to(self.device)
                '''
                n_samples = 20
                rec_loss = torch.tensor(0.0).to(self.device)
                for i in range(n_samples):
                    bows_recon,mus,log_vars = self.vae(bows,lambda x:torch.softmax(x,dim=1))
                    
                    logsoftmax = torch.log_softmax(bows_recon,dim=1)
                    _rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                    rec_loss += _rec_loss
                rec_loss = rec_loss / n_samples
                '''
                bows_recon,mus,log_vars = self.vae(bows,lambda x:torch.softmax(x,dim=1))
                if criterion=='cross_entropy':
                    logsoftmax = torch.log_softmax(bows_recon,dim=1)
                    rec_loss = -1.0 * torch.sum(bows*logsoftmax)
                elif criterion=='bce_softmax':
                    rec_loss = F.binary_cross_entropy(torch.softmax(bows_recon,dim=1),bows,reduction='sum')
                elif criterion=='bce_sigmoid':
                    rec_loss = F.binary_cross_entropy(torch.sigmoid(bows_recon),bows,reduction='sum')
                
                kl_div = -0.5 * torch.sum(1+log_vars-mus.pow(2)-log_vars.exp())
                
                loss = rec_loss + kl_div * beta
                
                loss.backward()
                optimizer.step()

                trainloss_lst.append(loss.item()/len(bows))
                epochloss_lst.append(loss.item()/len(bows))
                if (iter+1) % 10==0:
                    print(f'Epoch {(epoch+1):>3d}\tIter {(iter+1):>4d}\tLoss:{loss.item()/len(bows):<.7f}\tRec Loss:{rec_loss.item()/len(bows):<.7f}\tKL Div:{kl_div.item()/len(bows):<.7f}')
            #scheduler.step()
            if (epoch+1) % log_every==0:
                save_name = f'./ckpt/ETM_{self.taskname}_tp{self.n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}_ep{epoch+1}.ckpt'
                checkpoint = {
                    "net": self.vae.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "epoch": epoch,
                    "param": {
                        "bow_dim": self.bow_dim,
                        "n_topic": self.n_topic,
                        "taskname": self.taskname,
                        "emb_dim": self.emb_dim
                    }
                }
                torch.save(checkpoint,save_name)
                # The code lines between this and the next comment lines are duplicated with WLDA.py, consider to simpify them.
                print(f'Epoch {(epoch+1):>3d}\tLoss:{sum(epochloss_lst)/len(epochloss_lst):<.7f}')
                print('\n'.join([str(lst) for lst in self.show_topic_words()]))
                print('='*30)
                smth_pts = smooth_curve(trainloss_lst)
                plt.plot(np.array(range(len(smth_pts)))*log_every,smth_pts)
                plt.xlabel('epochs')
                plt.title('Train Loss')
                plt.savefig('gsm_trainloss.png')
                if test_data!=None:
                    c_v,c_w2v,c_uci,c_npmi,mimno_tc, td = self.evaluate(test_data,calc4each=False)
                    c_v_lst.append(c_v), c_w2v_lst.append(c_w2v), c_uci_lst.append(c_uci),c_npmi_lst.append(c_npmi), mimno_tc_lst.append(mimno_tc), td_lst.append(td)
                save_name = f'./ckpt/ETM_{self.taskname}_tp{self.n_topic}_{time.strftime("%Y-%m-%d-%H-%M", time.localtime())}.ckpt'
                torch.save(self.vae.state_dict(),save_name)
        scrs = {'c_v':c_v_lst,'c_w2v':c_w2v_lst,'c_uci':c_uci_lst,'c_npmi':c_npmi_lst,'mimno_tc':mimno_tc_lst,'td':td_lst}
        '''
        for scr_name,scr_lst in scrs.items():
            plt.cla()
            plt.plot(np.array(range(len(scr_lst)))*log_every,scr_lst)
            plt.savefig(f'wlda_{scr_name}.png')
        '''
        plt.cla()
        for scr_name,scr_lst in scrs.items():
            if scr_name in ['c_v','c_w2v','td']:
                plt.plot(np.array(range(len(scr_lst)))*log_every,scr_lst,label=scr_name)
        plt.title('Topic Coherence')
        plt.xlabel('epochs')
        plt.legend()
        plt.savefig(f'gsm_tc_scores.png')
        # The code lines between this and the last comment lines are duplicated with WLDA.py, consider to simpify them.


    def evaluate(self,test_data,calc4each=False):
        topic_words = self.show_topic_words()
        return evaluate_topic_quality(topic_words, test_data, taskname=self.taskname, calc4each=calc4each)


    def inference_by_bow(self,doc_bow):
        # doc_bow: torch.tensor [vocab_size]; optional: np.array [vocab_size]
        if isinstance(doc_bow,np.ndarray):
            doc_bow = torch.from_numpy(doc_bow)
        doc_bow = doc_bow.reshape(-1,self.bow_dim).to(self.device)
        with torch.no_grad():
            mu,log_var = self.vae.encode(doc_bow)
            mu = self.vae.fc1(mu) 
            theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()


    def inference(self, doc_tokenized, dictionary,normalize=True):
        doc_bow = torch.zeros(1,self.bow_dim)
        for token in doc_tokenized:
            try:
                idx = dictionary.token2id[token]
                doc_bow[0][idx] += 1.0
            except:
                print(f'{token} not in the vocabulary.')
        doc_bow = doc_bow.to(self.device)
        with torch.no_grad():
            mu,log_var = self.vae.encode(doc_bow)
            mu = self.vae.fc1(mu)
            if normalize:
                theta = F.softmax(mu,dim=1)
            return theta.detach().cpu().squeeze(0).numpy()

    def get_embed(self,train_data, num=1000):
        self.vae.eval()
        data_loader = DataLoader(train_data, batch_size=512,shuffle=False, num_workers=4, collate_fn=train_data.collate_fn)
        embed_lst = []
        txt_lst = []
        cnt = 0
        for data_batch in data_loader:
            txts, bows = data_batch
            embed = self.inference_by_bow(bows)
            embed_lst.append(embed)
            txt_lst.append(txts)
            cnt += embed.shape[0]
            if cnt>=num:
                break
        embed_lst = np.concatenate(embed_lst,axis=0)[:num]
        txt_lst = np.concatenate(txt_lst,axis=0)[:num]
        return txt_lst, embed_lst

    def get_topic_word_dist(self,normalize=True):
        self.vae.eval()
        with torch.no_grad():
            idxes = torch.eye(self.n_topic).to(self.device)
            word_dist = self.vae.decode(idxes)  # word_dist: [n_topic, vocab.size]
            if normalize:
                word_dist = F.softmax(word_dist,dim=1)
            return word_dist.detach().cpu().numpy()

    def show_topic_words(self,topic_id=None,topK=15, dictionary=None):
        topic_words = []
        idxes = torch.eye(self.n_topic).to(self.device)
        word_dist = self.vae.decode(idxes)
        word_dist = torch.softmax(word_dist,dim=1)
        vals,indices = torch.topk(word_dist,topK,dim=1)
        vals = vals.cpu().tolist()
        indices = indices.cpu().tolist()
        if self.id2token==None and dictionary!=None:
            self.id2token = {v:k for k,v in dictionary.token2id.items()}
        if topic_id==None:
            for i in range(self.n_topic):
                topic_words.append([self.id2token[idx] for idx in indices[i]])
        else:
            topic_words.append([self.id2token[idx] for idx in indices[topic_id]])
        return topic_words

    def load_model(self, model):
        self.vae.load_state_dict(model)



In [ ]:

model = EVAE(encode_dims=[1024,512,256,20],decode_dims=[20,128,768,1024],emb_dim=300)
model

In [ ]:
mod = ETM(dropout=0.0,num_topics = 20, vocab_size = len(dictionary), t_hidden_size = 800, emb_dim = 300,  
          embeddings=None, train_embeddings=True, drop=0.0)

mod